# FITS test

Here we load some real-world observations, and attempt to use our model trained on the simulated catalogue to output PDFs for the observations.

The first step is to open up the FITS catalogue, and have a peek inside

In [ ]:
import os
import logging
import numpy as np
import torch as t
import matplotlib.pyplot as plt

from astropy.io import fits

import agnfinder

from agnfinder import config as cfg
from agnfinder.prospector.load_photometry import get_filters

try:
    assert(_SETUP)
except NameError:
    cfg.configure_logging()
    os.chdir(os.path.split(agnfinder.__path__[0])[0])
    dtype = t.float32
    device = t.device("cuda") if t.cuda.is_available() else t.device("cpu")
    if device == t.device("cuda"):
        logging.info("Using GPU for models")
    else:
        logging.info("CUDA is unavailable; using CPU for models")
    _SETUP = True

In [ ]:
data_fpath: str = './data/DES_VIDEO_v1.0.1.fits'
hdul = fits.open(data_fpath)

In [ ]:
hdul.info()

In [ ]:
hdul[0].header

In [ ]:
hdul[1].columns

In [ ]:
hdul[1].header

In [ ]:
import h5py

In [ ]:
f = h5py.File('./data/cubes/des_sample/photometry_simulation_40000000n_z_0p0000_to_6p0000.hdf5')

In [ ]:
samples = f.get('samples')

In [ ]:
simulated_y = samples.get('simulated_y')

We need to make sure that the columns (i.e. filters) for the photometric measurements from the real-world observations are in the same order as the filters in the simulated catalogue.

In [ ]:
hdul[1].data.shape[0]

In [ ]:
hdul[1].columns

In [ ]:
hdul[1].data['ZAUTOMAG']

In [ ]:
from agnfinder.types import Filters

In [ ]:
Filters.DES.dim

Load up the mags columns of the dataset in the correct order:

NOTE: these are mags, not maggies. Need to convert them.

In [ ]:
cols = [hdul[1].data[f.mag_col][:,None] for f in get_filters('des')]
dset = np.concatenate(cols, 1)
assert dset.shape == (hdul[1].data.shape[0], Filters.DES.dim)